In [2]:
def getSbatch(n_systems, bh_kick, ns_kick, sigma_kick, run_size=10000, job_name='sse_evolution', nodes=None, queue='development'):
    output_path = f'$WORK/supernova_remnant/{job_name}'    
    
    n_systems = int(n_systems)
    run_size = int(run_size)
    n_runs = int(n_systems/run_size)
    
    if nodes is None:
        nodes = int(n_runs/56)+1
    tasks = min(56, n_runs)
            
    print(f"Generating script for {n_systems} stars using {n_runs} runs. \n")
  
    if queue=='development':
        timeout = '02:00:00'
    else:
        timeout = '10:00:00'
        
        
    preamble_text = f"""#!/bin/bash
## COSMIC Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J {job_name}           # Job name
#SBATCH -o {job_name}.o%j	   # Name of stdout output file
#SBATCH -e {job_name}.e%j	   # Name of stderr error file
#SBATCH -p {queue}          # Queue (partition) name
#SBATCH -N {nodes}             # Total # of nodes
#SBATCH -n {tasks}             # Total # of mpi tasks
#SBATCH -t {timeout}       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

mkdir {output_path}

# Launch MPI code...
    """
    
    if n_runs<1:
        raise ValueError('Total number of stars is less than run size')    
    
    if n_runs>512:
        raise ValueError('Total number of runs exceeds job limit')  
        
    commands_text = ""
    
    for i in range(n_runs):
        specs = f" --number-of-systems {run_size} --output-path {output_path}/ --mode SSE --kick-magnitude-distribution MULLERMANDEL --remnant-mass-prescription MULLERMANDEL --muller-mandel-kick-multiplier-BH {bh_kick} --muller-mandel-kick-multiplier-NS {ns_kick} --muller-mandel-sigma-kick {sigma_kick}"
        commands_text = commands_text + f"srun -N 1 -n 1 ~/supernova_remnant/COMPAS/src/COMPAS" + specs + f" --output-container run_{i} & \n"
    
    commands_text = commands_text + "wait"
    preamble_text += commands_text 
    
    return preamble_text



In [3]:
n_systems = 1e5
bh_kick = 200.0
ns_kick = 400.0
sigma_kick = 0.3
job_name = f'sn_rem_bh_{bh_kick}_ns_{ns_kick}_sigma_{sigma_kick}'

sbatch_text = getSbatch(n_systems, bh_kick, ns_kick, sigma_kick, job_name=job_name)
print(sbatch_text)

Generating script for 100000 stars using 10 runs. 

#!/bin/bash
## COSMIC Slurm job script
#   for TACC Frontera CLX nodes
# 
#   *** MPI Job in Development Queue ***
#
#----------------------------------------------------

#SBATCH -J sn_rem_bh_200.0_ns_400.0_sigma_0.3           # Job name
#SBATCH -o sn_rem_bh_200.0_ns_400.0_sigma_0.3.o%j	   # Name of stdout output file
#SBATCH -e sn_rem_bh_200.0_ns_400.0_sigma_0.3.e%j	   # Name of stderr error file
#SBATCH -p development          # Queue (partition) name
#SBATCH -N 1             # Total # of nodes
#SBATCH -n 10             # Total # of mpi tasks
#SBATCH -t 02:00:00       # Run time (hh:mm:ss)
#SBATCH --mail-type=all    # Send email at begin and end of job
#SBATCH --mail-user=vkapil1@jh.edu

# Any other commands must follow all #SBATCH directives...

# Load modules
module reset 
module load TACC intel impi hdf5 gsl gcc
# Print information
module list
pwd
date

mkdir $WORK/supernova_remnant/sn_rem_bh_200.0_ns_400.0_sigma_0.3

# Launch M

In [4]:
file1 = open(f"../sn_rem_sbatch/{job_name}", "w")  # write mode
file1.write(sbatch_text)
file1.close()